In [1]:
cd ..

/home/angran/GIT/xt-training


In [2]:
import os

import numpy as np
import torch
import torchvision
from PIL import Image
from torch import nn
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from xt_training import metrics
from xt_training.utils import (DummyOptimizer, SKDataset, SKInterface,
                               functional)

In [3]:
cd ../msg

/home/angran/GIT/msg


In [4]:
from utils import transforms as xt_transforms

/home/angran/anaconda3/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [5]:
# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has num_classes which is user-defined
num_classes = 2  # 1 class (person) + background

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

model.eager_outputs = lambda l, d: (l, d)

In [6]:
class RCNNDataset(object):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

    def __getitem__(self, idx):
        # load images and masks
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        mask = Image.open(mask_path)
        # convert the PIL Image into a numpy array
        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.imgs)

### Dataset

In [7]:
# our dataset has two classes only - background and person
num_classes = 2
# use our dataset and defined transformations
dataset = RCNNDataset(
    root='/home/angran/Documents/datasets/PennFudanPed', 
    transforms=transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor()
    ])
)
test_ds = RCNNDataset(
    root='/home/angran/Documents/datasets/PennFudanPed', 
    transforms=transforms.ToTensor()
)

# split the dataset in train and test set
indices = torch.randperm(len(dataset)).tolist()
train_ds = torch.utils.data.Subset(dataset, indices[:100])
valid_ds = torch.utils.data.Subset(dataset, indices[100:120])
test_ds = torch.utils.data.Subset(test_ds, indices[-50:])

### Dataloaders

In [8]:
def collate_fn(data):
    """
       data: is a list of tuples with (image, target)
             where 'example' is a tensor of arbitrary shape
             and target is a dictionary of tensor values
    """
    images, targets = zip(*data)
    
    return images, targets

In [9]:
# define training and validation data loaders
train_loader = DataLoader(
    train_ds, batch_size=1, shuffle=True, collate_fn=collate_fn)

valid_loader = DataLoader(
    valid_ds, batch_size=1, shuffle=True, collate_fn=collate_fn)

test_loaders = {}
test_loaders['test'] = DataLoader(
    test_ds, batch_size=1, shuffle=False, collate_fn=collate_fn)

In [10]:
# for i_batch, (x, y) in enumerate(test_loaders['test']):
#     break

In [11]:
# from collections.abc import Iterable

# if isinstance(x, torch.Tensor):
#     x = x
# elif isinstance(x, Iterable):
#     x = [x_i for x_i in x]

# if isinstance(y, torch.Tensor):
#     y = y.to(device)
# elif isinstance(y, Iterable):
#     y_tmp = []
#     for y_i in y:
#         if isinstance(y_i, dict):
#             y_i = {k: v for k, v in y_i.items()}
#         else:
#             y_i
#         y_tmp.append(y_i)
#     y = y_tmp

In [12]:
class ODHandler:
    def __init__(self, callable, default=None):
        self.fn = callable
        self.default = default
    def __call__(self, inputs, y):
        try:
            return self.fn(inputs, y)
        except:
            return self.default

In [13]:
loss_fn = ODHandler(lambda loss, y_pred: sum(l for l in loss.values()), True)

In [14]:
# fn = lambda loss, y_pred: sum(l for l in loss.values())

### Model & Optimizer

In [15]:
# move model to the right device
# model.to(device) # Already implemented in xt-training

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [16]:
len(train_loader), len(valid_loader), len(test_loaders['test'])

(100, 20, 50)

### Model Training

In [17]:
save_dir = '/home/angran/Downloads/test/'

# stats, matrix = 
functional.train(
    save_dir=save_dir,
    train_loader=train_loader,
    model=model,
    optimizer=optimizer,
    epochs=2,
    loss_fn=loss_fn,
    overwrite=True,
    val_loader=valid_loader,
    test_loaders=test_loaders,
#     device='cuda:0',
    scheduler=lr_scheduler,
#     eval_metrics=eval_metrics,
#     on_exit=default_train_exit
)

Running on device: cuda:0


Initial
----------


/home/angran/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


test     |    50/50   | loss:    0.0000 | eps:   16.6899   
valid    |    20/20   | loss:    0.0000 | eps:   18.3249   


/home/angran/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:3123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dtype=torch.float32)).float())) for i in range(dim)]
/home/angran/anaconda3/lib/python3.7/site-packages/torchvision/models/detection/anchor_utils.py:147: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(image_size[1] // g[1], dtype=torch.int64, device=device)] for g in grid_sizes]
/home/angran/anaconda3/lib/python3.7/site-packages/torch/tensor.py:593: RuntimeWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give i


Epoch 1/2
----------
train    |   100/100  | loss:    0.3321 | eps:    0.0000   
valid    |    20/20   | loss:    0.0000 | eps:   17.9753   

Epoch 2/2
----------
train    |   100/100  | loss:    0.2606 | eps:    0.0000   
valid    |    20/20   | loss:    0.0000 | eps:   17.7941   

Test
----------
test     |    50/50   | loss:    0.0000 | eps:   17.0683   


In [18]:
dddd

NameError: name 'dddd' is not defined

In [ ]:
for i_batch, (x, y) in enumerate(valid_loader):
    break
y[0]['boxes'].size()

In [ ]:
for i_batch, (x, y) in enumerate(test_loaders['test']):
    break

In [ ]:
y['boxes'].size()

In [ ]:
y[0]['boxes'].size()

In [ ]:
iterator = iter(train_loader)

In [ ]:
with torch.set_grad_enabled(False):
    x, y = next(iterator)

In [ ]:
y[0]['boxes'].size()

In [ ]:
dddd

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
 train_ds, batch_size=2, shuffle=True, num_workers=4, collate_fn=collate_fn
)
# For Training
images,targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images,targets)   # Returns losses and detections

In [ ]:
train_loader = DataLoader(
    train_ds, batch_size=1, shuffle=True)

In [ ]:
from collections.abc import Iterable

for i_batch, (x, y) in enumerate(train_loader):
    if isinstance(y, Iterable):
        print(type(y))
        print(y['boxes'].size())
    break

In [ ]:
x.size()

In [ ]:
train_loader.dataset[0][1]['boxes'].size()

In [ ]:
dir(train_loader)